In [1]:
# import local files with defined functions to acquire and prepare data
import env
import acquire
import prepare

# import libraries for visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# import libraries for data handling and formatting
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# import libraries for creating and evaluating models
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

ModuleNotFoundError: No module named 'env'